In [1]:
import torch
torch.cuda.set_device(1)
print('cuda:', torch.cuda.current_device())
device = "cuda" if torch.cuda.is_available() else "cpu"

cuda: 1


# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
segment_len = 128
d_model = 160
conform_dropout = 0.1
sa_dim = 48
model_name = 'HW04-conform-sa48_dmodel160'

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker neme to their corresponding id.
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append(
                    [utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if self.segment_len == -1:
            mel = torch.FloatTensor(mel)
        elif len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start + self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
            
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker

    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(
        mel, batch_first=True,
        padding_value=-20)  # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = myDataset(data_dir, segment_len)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
#         print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
        out = out.view(32, -1)      
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
    


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [6]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps))
        return max(
            0.0, 0.5 *
            (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [7]:
import torch


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [8]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset),
                ncols=0,
                desc="Valid",
                unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)

# Main function

In [9]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "save_path": model_name,
        "batch_size": 32,
        "n_workers": 2,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 120000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(
        data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!", flush=True)

    model = Classifier(n_spks=speaker_num).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps,
                                                total_steps)
    print(f"[Info]: Finish creating model!", flush=True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(
                f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})"
            )

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:29<00:00, 22.24 step/s, accuracy=0.59, loss=2.16, step=2000]
Valid: 100% 6944/6944 [00:02<00:00, 2690.37 uttr/s, accuracy=0.45, loss=2.51]
Train: 100% 2000/2000 [01:25<00:00, 23.45 step/s, accuracy=0.59, loss=1.43, step=4000]
Valid: 100% 6944/6944 [00:02<00:00, 2668.66 uttr/s, accuracy=0.62, loss=1.68]
Train: 100% 2000/2000 [01:30<00:00, 22.17 step/s, accuracy=0.66, loss=1.24, step=6000]
Valid: 100% 6944/6944 [00:02<00:00, 2699.78 uttr/s, accuracy=0.67, loss=1.41]
Train: 100% 2000/2000 [01:28<00:00, 22.71 step/s, accuracy=0.72, loss=1.34, step=8000]
Valid: 100% 6944/6944 [00:02<00:00, 2697.73 uttr/s, accuracy=0.72, loss=1.19]
Train: 100% 2000/2000 [01:25<00:00, 23.31 step/s, accuracy=0.88, loss=0.51, step=1e+4]
Valid: 100% 6944/6944 [00:02<00:00, 2634.52 uttr/s, accuracy=0.73, loss=1.15]
Train:   0% 4/2000 [00:00<02:16, 14.64 step/s, accuracy=0.78, loss=1.15, step=1e+4]

Step 10000, best model saved. (accuracy=0.7308)


Train: 100% 2000/2000 [01:28<00:00, 22.49 step/s, accuracy=0.81, loss=0.69, step=12000]
Valid: 100% 6944/6944 [00:02<00:00, 2771.18 uttr/s, accuracy=0.76, loss=1.07]
Train: 100% 2000/2000 [01:29<00:00, 22.46 step/s, accuracy=0.88, loss=0.64, step=14000]
Valid: 100% 6944/6944 [00:02<00:00, 2618.88 uttr/s, accuracy=0.77, loss=0.97]
Train: 100% 2000/2000 [01:31<00:00, 21.94 step/s, accuracy=0.84, loss=0.50, step=16000]
Valid: 100% 6944/6944 [00:02<00:00, 2608.50 uttr/s, accuracy=0.79, loss=0.94]
Train: 100% 2000/2000 [01:26<00:00, 23.06 step/s, accuracy=0.78, loss=0.71, step=18000]
Valid: 100% 6944/6944 [00:02<00:00, 2594.53 uttr/s, accuracy=0.79, loss=0.91]
Train: 100% 2000/2000 [01:27<00:00, 22.86 step/s, accuracy=0.97, loss=0.16, step=2e+4] 
Valid: 100% 6944/6944 [00:02<00:00, 2762.98 uttr/s, accuracy=0.80, loss=0.86]
Train:   0% 4/2000 [00:00<02:24, 13.80 step/s, accuracy=0.88, loss=0.33, step=2e+4]

Step 20000, best model saved. (accuracy=0.8033)


...

Step 100000, best model saved. (accuracy=0.9075)


Train: 100% 2000/2000 [01:27<00:00, 22.82 step/s, accuracy=1.00, loss=0.00, step=102000]
Valid: 100% 6944/6944 [00:02<00:00, 2453.36 uttr/s, accuracy=0.91, loss=0.46]
Train: 100% 2000/2000 [01:29<00:00, 22.24 step/s, accuracy=1.00, loss=0.00, step=104000]
Valid: 100% 6944/6944 [00:02<00:00, 2763.33 uttr/s, accuracy=0.91, loss=0.46]
Train: 100% 2000/2000 [01:28<00:00, 22.70 step/s, accuracy=1.00, loss=0.01, step=106000]
Valid: 100% 6944/6944 [00:02<00:00, 2555.68 uttr/s, accuracy=0.91, loss=0.44]
Train: 100% 2000/2000 [01:29<00:00, 22.44 step/s, accuracy=1.00, loss=0.01, step=108000]
Valid: 100% 6944/6944 [00:02<00:00, 2641.09 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [01:26<00:00, 23.18 step/s, accuracy=1.00, loss=0.00, step=110000]
Valid: 100% 6944/6944 [00:02<00:00, 2737.96 uttr/s, accuracy=0.91, loss=0.45]
Train:   0% 4/2000 [00:00<02:02, 16.33 step/s, accuracy=1.00, loss=0.01, step=110004]

Step 110000, best model saved. (accuracy=0.9129)


Train: 100% 2000/2000 [01:25<00:00, 23.36 step/s, accuracy=1.00, loss=0.01, step=112000]
Valid: 100% 6944/6944 [00:02<00:00, 2742.49 uttr/s, accuracy=0.91, loss=0.46]
Train: 100% 2000/2000 [01:26<00:00, 23.20 step/s, accuracy=1.00, loss=0.00, step=114000]
Valid: 100% 6944/6944 [00:02<00:00, 2591.07 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [01:28<00:00, 22.70 step/s, accuracy=1.00, loss=0.01, step=116000]
Valid: 100% 6944/6944 [00:02<00:00, 2642.55 uttr/s, accuracy=0.91, loss=0.47]
Train: 100% 2000/2000 [01:28<00:00, 22.53 step/s, accuracy=1.00, loss=0.00, step=118000]
Valid: 100% 6944/6944 [00:02<00:00, 2600.96 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [01:26<00:00, 23.10 step/s, accuracy=1.00, loss=0.00, step=120000]
Valid: 100% 6944/6944 [00:02<00:00, 2666.68 uttr/s, accuracy=0.91, loss=0.45]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 120000, best model saved. (accuracy=0.9146)


# Inference

## Dataset of inference

In [10]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


def inference_collate_batch(batch):
    """Collate a batch of data."""
    feat_paths, mels = zip(*batch)

    return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [11]:
# import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
        print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
#         out = out.view(-1, d_model)  
        out = out.view(32, -1)  
        print(out.shape)
        print(sa_dim)
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
a = torch.randn(32,150,40)
model = Classifier()
# print(model)
model(a).shape

A torch.Size([32, 48, 150])
torch.Size([32, 7680])
48


torch.Size([32, 600])

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
        print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
#         out = out.view(-1, d_model)  
        out = out.view(1, -1)  
        print(out.shape)
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
a = torch.randn(1,150,40)
model = Classifier()
# print(model)
model(a).shape

A torch.Size([1, 48, 150])
torch.Size([1, 7680])


torch.Size([1, 600])

In [13]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader


def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "model_path": model_name,
        "output_path": "./output_{}.csv".format(model_name),
    }

    return config


def main(
    data_dir,
    model_path,
    output_path,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=8,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!", flush=True)

    speaker_num = len(mapping["id2speaker"])
    model = Classifier(n_spks=speaker_num).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"[Info]: Finish creating model!", flush=True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            print(mels.shape)
            outs = model(mels)
            print(outs.shape)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/6000 [00:00<?, ?it/s]

torch.Size([1, 2112, 40])
A torch.Size([1, 48, 2112])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 586, 40])
A torch.Size([1, 48, 586])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 465, 40])
A torch.Size([1, 48, 465])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 404, 40])
A torch.Size([1, 48, 404])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 402, 40])
A torch.Size([1, 48, 402])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 428, 40])
A torch.Size([1, 48, 428])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 899, 40])
A torch.Size([1, 48, 899])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1677, 40])
A torch.Size([1, 48, 1677])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 335, 40])
A torch.Size([1, 48, 335])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 528, 40])
A torch.Size([1, 48, 528])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 533, 40])
A torch.Size([1, 48, 

torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 659, 40])
A torch.Size([1, 48, 659])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 519, 40])
A torch.Size([1, 48, 519])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 493, 40])
A torch.Size([1, 48, 493])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 377, 40])
A torch.Size([1, 48, 377])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 669, 40])
A torch.Size([1, 48, 669])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 387, 40])
A torch.Size([1, 48, 387])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 531, 40])
A torch.Size([1, 48, 531])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 574, 40])
A torch.Size([1, 48, 574])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 512, 40])
A torch.Size([1, 48, 512])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1090, 40])
A torch.Size([1, 48, 1090])
torch.Size([1, 7680])
torch.Size([1, 600])
torch

torch.Size([1, 408, 40])
A torch.Size([1, 48, 408])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 512, 40])
A torch.Size([1, 48, 512])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 405, 40])
A torch.Size([1, 48, 405])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 508, 40])
A torch.Size([1, 48, 508])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 851, 40])
A torch.Size([1, 48, 851])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 492, 40])
A torch.Size([1, 48, 492])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 627, 40])
A torch.Size([1, 48, 627])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1105, 40])
A torch.Size([1, 48, 1105])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 442, 40])
A torch.Size([1, 48, 442])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 396, 40])
A torch.Size([1, 48, 396])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 739, 40])
A torch.Size([1, 48, 73

...

torch.Size([1, 985, 40])
A torch.Size([1, 48, 985])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 364, 40])
A torch.Size([1, 48, 364])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 401, 40])
A torch.Size([1, 48, 401])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 827, 40])
A torch.Size([1, 48, 827])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 464, 40])
A torch.Size([1, 48, 464])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 513, 40])
A torch.Size([1, 48, 513])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 521, 40])
A torch.Size([1, 48, 521])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 493, 40])
A torch.Size([1, 48, 493])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1707, 40])
A torch.Size([1, 48, 1707])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1255, 40])
A torch.Size([1, 48, 1255])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 707, 40])
A torch.Size([1, 48, 

A torch.Size([1, 48, 472])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 948, 40])
A torch.Size([1, 48, 948])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 576, 40])
A torch.Size([1, 48, 576])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 772, 40])
A torch.Size([1, 48, 772])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 470, 40])
A torch.Size([1, 48, 470])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 316, 40])
A torch.Size([1, 48, 316])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 552, 40])
A torch.Size([1, 48, 552])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 956, 40])
A torch.Size([1, 48, 956])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 552, 40])
A torch.Size([1, 48, 552])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 907, 40])
A torch.Size([1, 48, 907])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 509, 40])
A torch.Size([1, 48, 509])
torch.Size([1, 7680])
t

A torch.Size([1, 48, 421])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1769, 40])
A torch.Size([1, 48, 1769])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 906, 40])
A torch.Size([1, 48, 906])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 716, 40])
A torch.Size([1, 48, 716])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 308, 40])
A torch.Size([1, 48, 308])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 424, 40])
A torch.Size([1, 48, 424])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 646, 40])
A torch.Size([1, 48, 646])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 781, 40])
A torch.Size([1, 48, 781])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 460, 40])
A torch.Size([1, 48, 460])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 408, 40])
A torch.Size([1, 48, 408])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 411, 40])
A torch.Size([1, 48, 411])
torch.Size([1, 7680])

torch.Size([1, 1240, 40])
A torch.Size([1, 48, 1240])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 664, 40])
A torch.Size([1, 48, 664])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 580, 40])
A torch.Size([1, 48, 580])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 649, 40])
A torch.Size([1, 48, 649])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 393, 40])
A torch.Size([1, 48, 393])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 478, 40])
A torch.Size([1, 48, 478])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 334, 40])
A torch.Size([1, 48, 334])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 648, 40])
A torch.Size([1, 48, 648])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1265, 40])
A torch.Size([1, 48, 1265])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 470, 40])
A torch.Size([1, 48, 470])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 500, 40])
A torch.Size([1, 48, 

torch.Size([1, 1418, 40])
A torch.Size([1, 48, 1418])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 524, 40])
A torch.Size([1, 48, 524])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1051, 40])
A torch.Size([1, 48, 1051])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 473, 40])
A torch.Size([1, 48, 473])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 402, 40])
A torch.Size([1, 48, 402])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 924, 40])
A torch.Size([1, 48, 924])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 756, 40])
A torch.Size([1, 48, 756])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 344, 40])
A torch.Size([1, 48, 344])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 1007, 40])
A torch.Size([1, 48, 1007])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 350, 40])
A torch.Size([1, 48, 350])
torch.Size([1, 7680])
torch.Size([1, 600])
torch.Size([1, 721, 40])
A torch.Size([1, 48